In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import os
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import gc
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
#scaler = MinMaxScaler()  => Standard is better here

path = './BandMyo_data/'  

In [3]:
def data_processing(subject):
    lst_len = []
    for label in range(1,10):   #1~9
        tmp = glob.glob(path + '/00{}/00{}/*.mat'.format(subject, label))
        for gesture in range(len(tmp)):
            tmp2 = scipy.io.loadmat(tmp[gesture])
            globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label, gesture)] = tmp2['data'].T
            for i in range(8):
                lst_len.append(len(globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label, gesture)][i]))


    for label in range(6):  #10~15
        tmp = glob.glob(path + '/00{}/01{}/*.mat'.format(subject, label))
        for gesture in range(len(tmp)):
            tmp2 = scipy.io.loadmat(tmp[gesture])
            globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label + 10, gesture)] = tmp2['data'].T
            for i in range(8):
                lst_len.append(len(globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label + 10, gesture)][i]))

    length = np.min(lst_len)
    tmp_lst, labels = [], []
                    

    for label in range(1,16):
        for gesture in range(8):
            for idx in range(3): 
                for sensors in range(8):
                    tm = globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label, gesture)][sensors]
                    tmp0 = pd.DataFrame(tm)
                    tmp = tmp0.sample(n=length)
                    new_tmp = tmp.sort_index().values.tolist()
                    new_tmp_ = new_tmp[200*idx:200*(idx+1)]
                    
                    new_tmp2 = scaler.fit_transform(new_tmp_)
                    tmp_lst.append(list(new_tmp2.reshape(-1)))
                    #tmp_lst.append(new_tmp_)
                    labels.append(label-1)

    for_cols = []
    for i in range(200): for_cols.append('t{}'.format(i))

    df = pd.DataFrame(tmp_lst)
    df.columns = for_cols
    df['Label'] = labels

    #train_idx = int(len(df)/40)*30
    #train_data = df.iloc[:train_idx, :]
    train_data = df.sample(frac=1)

    feature_names = set(df.columns) - {"Label"}
    num_features = len(feature_names)

    # Create train and test features as a numpy array.
    x_train = train_data[feature_names].to_numpy()
    y_train = train_data["Label"]

    node_features = tf.cast(
        df[feature_names].to_numpy(), dtype=tf.dtypes.float32
    )
    x_train = train_data.index.to_numpy()
    
    return x_train, y_train, node_features, df

In [4]:
def graph_build(df):
    src, dst = [], []
    
    for i in range(len(df)):
        src.append(i)
        if (i+1)%8 == 0: dst.append(i-7)
        else: dst.append(i+1)
        
    graph1 = pd.DataFrame([src, dst]).T
    graph1.columns = ['source', 'target']
    
    edges = graph1[["source", "target"]].to_numpy()#.T
    edge_weights = tf.ones(shape=edges.shape[1])
    
    return graph1, edges, edge_weights

In [5]:
class GraphAttention(layers.Layer):
    def __init__(
        self,
        units,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

    def build(self, input_shape):

        self.kernel = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel",
        )
        self.kernel_attention = self.add_weight(
            shape=(self.units * 2, 1),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel_attention",
        )
        self.built = True

    def call(self, inputs):
        node_states, edges = inputs

        node_states_transformed = tf.matmul(node_states, self.kernel)

        node_states_expanded = tf.gather(node_states_transformed, edges)
        node_states_expanded = tf.reshape(
            node_states_expanded, (tf.shape(edges)[0], -1)
        )
        attention_scores = tf.nn.leaky_relu(
            tf.matmul(node_states_expanded, self.kernel_attention)
        )
        attention_scores = tf.squeeze(attention_scores, -1)

        attention_scores = tf.math.exp(tf.clip_by_value(attention_scores, -2, 2))
        attention_scores_sum = tf.math.unsorted_segment_sum(
            data=attention_scores,
            segment_ids=edges[:, 0],
            num_segments=tf.reduce_max(edges[:, 0]) + 1,
        )
        attention_scores_sum = tf.repeat(
            attention_scores_sum, tf.math.bincount(tf.cast(edges[:, 0], "int32"))
        )
        attention_scores_norm = attention_scores / attention_scores_sum

        node_states_neighbors = tf.gather(node_states_transformed, edges[:, 1])
        out = tf.math.unsorted_segment_sum(
            data=node_states_neighbors * attention_scores_norm[:, tf.newaxis],
            segment_ids=edges[:, 0],
            num_segments=tf.shape(node_states)[0],
        )
        return out


class MultiHeadGraphAttention(layers.Layer):
    def __init__(self, units, num_heads=8, merge_type="concat", **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.merge_type = merge_type
        self.attention_layers = [GraphAttention(units) for _ in range(num_heads)]

    def call(self, inputs):
        atom_features, pair_indices = inputs
        outputs = [
            attention_layer([atom_features, pair_indices])
            for attention_layer in self.attention_layers
        ]
        if self.merge_type == "concat":
            outputs = tf.concat(outputs, axis=-1)
        else:
            outputs = tf.reduce_mean(tf.stack(outputs, axis=-1), axis=-1)
        return tf.nn.relu(outputs)

In [6]:
def plot(result):
    loss = result.history["loss"]
    acc = result.history["acc"]
    val_loss = result.history["val_loss"]
    val_acc = result.history["val_acc"]

    plt.figure(figsize=(13,4))

    plt.subplot(1,2,1)
    plt.plot(range(len(loss)),loss,label = "Train Loss", marker='o', markersize=3)
    plt.plot(range(len(val_loss)),val_loss,label = "Validation Loss", marker='o', markersize=3)
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.grid()
    plt.legend(loc='upper right')

    plt.subplot(1,2,2)
    plt.plot(range(len(acc)),acc,label = "Train Accuracy", marker='o', markersize=3)
    plt.plot(range(len(val_acc)),val_acc,label = "Validation Accuracy", marker='o', markersize=3)
    plt.title('Model Acc')
    plt.ylabel('Acc')
    plt.xlabel('Epoch')
    plt.grid()
    plt.legend(loc='upper left')
    plt.show()

In [7]:
class GraphAttentionNetwork(keras.Model):
    def __init__(
        self,
        node_states,
        edges,
        hidden_units,
        num_heads,
        num_layers,
        output_dim,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.node_states = node_states
        self.edges = edges
        self.preprocess = layers.Dense(hidden_units * num_heads, activation="relu")
        self.attention_layers = [
            MultiHeadGraphAttention(hidden_units, num_heads) for _ in range(num_layers)
        ]
        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs):
        node_states, edges = inputs
        x = self.preprocess(node_states)
        for attention_layer in self.attention_layers:
            x = attention_layer([x, edges]) + x
        outputs = self.output_layer(x)
        return outputs

    def train_step(self, data):
        indices, labels = data

        with tf.GradientTape() as tape:
            outputs = self([self.node_states, self.edges])
            loss = self.compiled_loss(labels, tf.gather(outputs, indices))
        
        grads = tape.gradient(loss, self.trainable_weights)
        optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.compiled_metrics.update_state(labels, tf.gather(outputs, indices))

        return {m.name: m.result() for m in self.metrics}

    def predict_step(self, data):
        indices = data
        outputs = self([self.node_states, self.edges])

        return tf.nn.softmax(tf.gather(outputs, indices))

    def test_step(self, data):
        indices, labels = data
        outputs = self([self.node_states, self.edges])
        loss = self.compiled_loss(labels, tf.gather(outputs, indices))
        self.compiled_metrics.update_state(labels, tf.gather(outputs, indices))

        return {m.name: m.result() for m in self.metrics}

In [9]:
def build_data(subject, LST_sensor):
    scaler = StandardScaler()
    
    lst_len = []
    for label in range(1,10):   #1~9
        tmp = glob.glob(path + '/00{}/00{}/*.mat'.format(subject, label))
        for gesture in range(len(tmp)):
            tmp2 = scipy.io.loadmat(tmp[gesture])
            globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label, gesture)] = tmp2['data'].T
            for i in range(8):
                lst_len.append(len(globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label, gesture)][i]))


    for label in range(6):  #10~15
        tmp = glob.glob(path + '/00{}/01{}/*.mat'.format(subject, label))
        for gesture in range(len(tmp)):
            tmp2 = scipy.io.loadmat(tmp[gesture])
            globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label + 10, gesture)] = tmp2['data'].T
            for i in range(8):
                lst_len.append(len(globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label + 10, gesture)][i]))

    length = np.min(lst_len)
    tmp_lst, labels = [], []


    for label in range(1,16):
        for gesture in range(8):
            for idx in range(3): 
                for sensors in LST_sensor:
                    tm = globals()['emg_sub{}_label{}_Gesture{}'.format(subject, label, gesture)][sensors]
                    tmp0 = pd.DataFrame(tm)
                    tmp = tmp0.sample(n=length)
                    new_tmp = tmp.sort_index().values.tolist()
                    new_tmp_ = new_tmp[200*idx:200*(idx+1)]

                    new_tmp2 = scaler.fit_transform(new_tmp_)
                    tmp_lst.append(list(np.abs(new_tmp2.reshape(-1))))  ######진짜 신기한게 절대값씌워서 돌리는게 학습 더 잘됌
                    #tmp_lst.append(list(new_tmp2.reshape(-1))) 
                    labels.append(label-1)

    for_cols = []
    for i in range(200): for_cols.append('t{}'.format(i))

    df = pd.DataFrame(tmp_lst)
    df.columns = for_cols
    df['Label'] = labels
    
    train_data = df.sample(frac=1)

    feature_names = set(df.columns) - {"Label"}
    num_features = len(feature_names)

    x_train = train_data[feature_names].to_numpy()
    y_train = train_data["Label"]

    node_features = tf.cast(df[feature_names].to_numpy(), dtype=tf.dtypes.float32)
    x_train = train_data.index.to_numpy()

    return x_train, y_train, node_features, df

In [10]:
HIDDEN_UNITS = 50
NUM_HEADS = 8
NUM_LAYERS = 3
OUTPUT_DIM = 15 #len(class_values)

NUM_EPOCHS = 200
BATCH_SIZE = 128 #128
val_split = 0.2
LEARNING_RATE = 0.001 #0.001
MOMENTUM = 0.9
class_values = 15

# All sensors

In [11]:
def prepare(lst_sensor, subject):
    x_train, y_train, node_features, df = build_data(subject, lst_sensor)
    graph1, edges, edge_weights = graph_build(df)
    graph_info = (node_features, edges, edge_weights) # Create graph info tuple with node_features, edges, and edge_weights.
    print(no_sensor, "   Edges shape:", edges.shape, "    Nodes shape:", node_features.shape)

    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = keras.optimizers.Adam(LEARNING_RATE)
    accuracy_fn = keras.metrics.SparseCategoricalAccuracy(name="acc")

    model_individual = GraphAttentionNetwork(node_features, edges, HIDDEN_UNITS, NUM_HEADS, NUM_LAYERS, OUTPUT_DIM)

    model_individual.compile(loss=loss_fn, optimizer=optimizer, metrics=[accuracy_fn])
    
    return model_individual, x_train, y_train, optimizer

In [ ]:
import gc, random
subject = 4
store_path = 'C:/Users/hml76/Desktop/Jupyter/Paper1__renew/New/BandMyo_data//Subject{}/'.format(subject)

 
for i in range(8): globals()['history_no_{}'.format(i)] = []

all_lst = []
for no_sensor in range(8):
    lst_sensor = []
    for i in range(8): 
        if i == no_sensor: pass
        else: lst_sensor.append(i)
    all_lst.append(lst_sensor)

five_lsts = random.sample(all_lst, 5)

for i in range(len(five_lsts)):
    model_individual, x_train, y_train, optimizer = prepare(five_lsts[i], subject)
    val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
    history_no_1.append(val)
    del model_individual
    gc.collect()

all_lst = []
for no_sensor in range(8):
    for i in range(no_sensor, 8): 
        lst_sensor = []
        for j in range(8):
            if i != no_sensor: 
                if j == no_sensor: pass 
                elif j == i: pass
                else: lst_sensor.append(j)
        
        if lst_sensor: #lst_sensor is not empty list []
            all_lst.append(lst_sensor)
            
five_lsts = random.sample(all_lst, 5)
for i in range(len(five_lsts)):
    model_individual, x_train, y_train, optimizer = prepare(five_lsts[i], subject)
    val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
    history_no_2.append(val)
    del model_individual
    gc.collect()

In [15]:
NUM_SELECT = 7

all_lst = []
for no_sensor in range(8):
    for i in range(no_sensor, 8): 
        for k in range(i, 8):
            lst_sensor = []
            for j in range(8):
                if i != no_sensor and i != k: 
                    if j == no_sensor: pass 
                    elif j == i: pass
                    elif j == k: pass
                    else: lst_sensor.append(j)
        
            if lst_sensor: #lst_sensor is not empty list []
                all_lst.append(lst_sensor)

five_lsts = random.sample(all_lst, NUM_SELECT)
for i in range(len(five_lsts)):
    model_individual, x_train, y_train, optimizer = prepare(five_lsts[i], subject)
    val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
    history_no_3.append(val)
    del model_individual
    gc.collect()

print("Step 3\n===============================================================\n")

all_lst = []
for no_sensor in range(8):
    for i in range(no_sensor, 8): 
        for k in range(i, 8):
            for p in range(k, 8):
                lst_sensor = []
                for j in range(8):
                    if i != no_sensor and i != k and i != p: 
                        if j == no_sensor: pass 
                        elif j == i: pass
                        elif j == k: pass
                        elif j == p: pass
                        else: 
                            lst_sensor.append(j)
                if lst_sensor and  len(lst_sensor) < 5:            
                    all_lst.append(lst_sensor)

five_lsts = random.sample(all_lst, NUM_SELECT)
for i in range(len(five_lsts)):
    model_individual, x_train, y_train, optimizer = prepare(five_lsts[i], subject)
    val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
    history_no_4.append(val)
    del model_individual
    gc.collect()

print("Step 4\n===============================================================\n")    
    
all_lst = []                    
for no_sensor in range(8):
    for i in range(no_sensor, 8): 
        for k in range(i, 8):
            for p in range(k, 8):
                for q in range(p, 8):
                    lst_sensor = []
                    for j in range(8):
                        if i != no_sensor and i != k and i != p and i != q: 
                            if j == no_sensor: pass 
                            elif j == i: pass
                            elif j == k: pass
                            elif j == p: pass
                            elif j == q: pass
                            else: lst_sensor.append(j)
                                
                    if lst_sensor and  len(lst_sensor) < 4:            
                        all_lst.append(lst_sensor)

five_lsts = random.sample(all_lst, NUM_SELECT)
for i in range(len(five_lsts)):
    model_individual, x_train, y_train, optimizer = prepare(five_lsts[i], subject)
    val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
    history_no_5.append(val)
    del model_individual
    gc.collect()

print("Step 5\n===============================================================\n")

all_lst = []
for no_sensor in range(8):
    for i in range(no_sensor, 8): 
        for k in range(i, 8):
            for p in range(k, 8):
                for q in range(p, 8):
                    for z in range(q, 8):
                        lst_sensor = []
                        for j in range(8):
                            if i != no_sensor and i != k and i != p and i != q and i != z: 
                                if j == no_sensor: pass 
                                elif j == i: pass
                                elif j == k: pass
                                elif j == p: pass
                                elif j == q: pass
                                elif j == z: pass
                                else: lst_sensor.append(j)
                        if lst_sensor and  len(lst_sensor) < 3:            
                            all_lst.append(lst_sensor)

five_lsts = random.sample(all_lst, NUM_SELECT)
for i in range(len(five_lsts)):
    model_individual, x_train, y_train, optimizer = prepare(five_lsts[i], subject)
    val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
    history_no_6.append(val)
    del model_individual
    gc.collect()

print("Step 6\n===============================================================\n")
    
all_lst = []
for no_sensor in range(8):
    lst_sensor = [no_sensor]
    all_lst.append(lst_sensor)
    
five_lsts = random.sample(all_lst, NUM_SELECT)
for i in range(len(five_lsts)):
    model_individual, x_train, y_train, optimizer = prepare(five_lsts[i], subject)
    val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
    history_no_7.append(val)
    del model_individual
    gc.collect()

7    Edges shape: (1800, 2)     Nodes shape: (1800, 200)
7    Edges shape: (1800, 2)     Nodes shape: (1800, 200)
7    Edges shape: (1800, 2)     Nodes shape: (1800, 200)
7    Edges shape: (1800, 2)     Nodes shape: (1800, 200)
7    Edges shape: (1800, 2)     Nodes shape: (1800, 200)
7    Edges shape: (1800, 2)     Nodes shape: (1800, 200)
7    Edges shape: (1800, 2)     Nodes shape: (1800, 200)
Step 3

7    Edges shape: (1440, 2)     Nodes shape: (1440, 200)
7    Edges shape: (1440, 2)     Nodes shape: (1440, 200)
7    Edges shape: (1440, 2)     Nodes shape: (1440, 200)
7    Edges shape: (1440, 2)     Nodes shape: (1440, 200)
7    Edges shape: (1440, 2)     Nodes shape: (1440, 200)
7    Edges shape: (1440, 2)     Nodes shape: (1440, 200)
7    Edges shape: (1440, 2)     Nodes shape: (1440, 200)
Step 4

7    Edges shape: (1080, 2)     Nodes shape: (1080, 200)
7    Edges shape: (1080, 2)     Nodes shape: (1080, 200)
7    Edges shape: (1080, 2)     Nodes shape: (1080, 200)
7    Edges shap

In [17]:
store_path

'C:/Users/hml76/Desktop/Jupyter/Paper1__renew/New/BandMyo_data//Subject4/'

In [20]:
model_individual, x_train, y_train, optimizer = prepare([0,1,2,3,4,5,6,7], subject)
val = model_individual.fit(x_train, y_train, validation_split=val_split, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
history_no_0.append(val)
del model_individual
gc.collect()

7    Edges shape: (2880, 2)     Nodes shape: (2880, 200)


2226

In [21]:
store_path = 'C:/Users/hml76/Desktop/Jupyter/Paper1__renew/New/performance/Subject4/'

for i in range(8):
    ACC, LOSS, VAL_ACC, VAL_LOSS = [], [], [], []
    
    data = globals()['history_no_{}'.format(i)]
    for j in range(len(data)):
        VAL_ACC.append(data[j].history['val_acc'])
        VAL_LOSS.append(data[j].history['val_loss'])
        ACC.append(data[j].history['acc'])
        LOSS.append(data[j].history['loss'])
        
    pd.DataFrame(VAL_ACC).to_csv(store_path+'missing_{}_sensors_val_acc.csv'.format(i)) 
    pd.DataFrame(VAL_LOSS).to_csv(store_path+'missing_{}_sensors_val_loss.csv'.format(i)) 
    pd.DataFrame(ACC).to_csv(store_path+'missing_{}_sensors_training_acc.csv'.format(i)) 
    pd.DataFrame(LOSS).to_csv(store_path+'missing_{}_sensors_training_loss.csv'.format(i)) 